# implement Gradient Descent Method

1. the focus is achieving random gradient descent method.
2. 在矩阵因式分解的过程中，为了是得目标函数求得极值，需要将其导数等于0，来找到极值点。
3. 为了解决在矩阵分解中两个（或者多个矩阵）同时变化的情况，这里使用的是交替最小二乘法来完成梯度下降的计算。交替最小二乘法ALS的核心思想是：
   1. 从多个变量，选定一个变量；
   2. 其他的变量都固定住（可以初始化一个值，这个初始化值非常有技术含量，初始化值可以通过多次随机来避免陷入局部最优解的情况），
   3. 然后对选定的变量的进行最小二乘法；
   4. 然后将该选定的变量进行最小二乘法之后的结果固定住；
   5. 再从余下的变量中再选一个变量；
   6. 重复2-5步骤，直到达到目标函数的要求。
4. 这里为什么要用梯度下降来求矩阵分解？是因为在3中需要求矩阵的逆。一般认为矩阵的逆是不好计算的。所以通过梯度下降来迭代求出矩阵的分解因子。这样就不需要目标函数的导数等于0了，只需要求偏导数乘以步长，来进行迭代即可。

In [1]:
import numpy as np

In [2]:
R = np.array([[4,0,1,0,5],
     [1,2,1,3,5],
     [4,5,3,1,0],
     [2,3,0,2,5],
     [5,1,4,0,0],
     [0,3,2,4,1]])
# 为0的地方并不是评分为0，而是用户并没有对该物品进行评价。没有评分的地方并不用考虑它的误差。
R.shape

(6, 5)

In [3]:
"""_summary_
输入
R 是m*n的评分矩阵
K 隐性特征向量维度。注意这个特征维度是人为定义的。但是按道理这不应该有人为定义。
steps/max_iter 最大迭代步长
alpha 步长
lamda 正则化系数

输出
分解之后的P和Q
P 初始化用户特征矩阵m*k
Q 初始化物品特征矩阵k*n
"""

# 对超参数进行赋值
K=2
max_iter = 5000 #迭代次数多意味着步长比较小。
alpha = 0.0002
lamda = 0.004

def grad(R, K=2, max_iter= 5000, alpha=0.001, lamda= 0.002, cost_threshold = 0.0001):
    m = len(R)
    n = len(R[0])
    
    P = np.random.rand(m, K)
    Q = np.random.rand(K, n)
    
    for step in range(max_iter):
        # 对所有的用户u和物品i做遍历。对对应的Pu和Qi向量进行梯度下降。
        for u in range(m):
            for i in range(n):
                # 对于每一个大于0的评分，求出评分误差。
                if R[u][i] > 0:
                    eui = np.dot(P[u, :],Q[:, i]) - R[u,i]
                    
                    # 带入梯度下降的公式，按照梯度下降算法更新当前的Pu和Qi。也就是按照K个隐藏维度来更新。
                    for k in range(K):
                        # 注意这里和公式不同的地方在于求和公式。由于求和是对i在求和，而本计算是包含在
                        # for i in range(n):当中的，就相对于每个步骤都减去了一个对于i的元素，所以不
                        # 用再求和了。
                        P[u][k] = P[u][k] - alpha * (2 * eui * Q[k][i] - 2 * lamda * P[u][k])
                        # 同样的
                        Q[k][i] = Q[k][i] - alpha * (2 * eui * P[u][k] - 2 * lamda * Q[k][i])
                
        # u和i遍历完成。所有特征向量都更新完成。可以计算预测评分矩阵。
        # predictR = np.dot(P, Q)
        # 计算当前的损失函数。
        cost = 0
        
        for u in range(m):
            for i in range(n):
                # 在评分矩阵R中为0的不计算损失函数，原因依然是为0的评分可能是用户没有评分。
                if R[u][i] > 0:
                    cost += (np.dot(P[u, :],Q[:, i]) - R[u,i]) ** 2
                    for k in range(K):
                        cost += lamda * (P[u][k] ** 2 + Q[k][i] ** 2)
        # 当损失函数小于某一个特定阈值时退出。
        if cost < cost_threshold:
            break
    return P, Q, cost

In [4]:
P, Q, cost = grad(R, K, max_iter, alpha, lamda)
predictR = np.dot(P, Q)
print("origin R is {}, \n\n predict Matrix is {}, \n\n  User matrix is {}, \n\n Item matrix is {}, \n\n Cost is {}\n\n".format(R, predictR, P, Q, cost))

origin R is [[4 0 1 0 5]
 [1 2 1 3 5]
 [4 5 3 1 0]
 [2 3 0 2 5]
 [5 1 4 0 0]
 [0 3 2 4 1]], 

 predict Matrix is [[ 3.63222493  4.31853327  1.58468883  3.203653    5.06734483]
 [ 1.39393529  2.84073958  0.22371082  2.08868786  4.85299239]
 [ 4.34674076  3.7001604   2.37327996  2.76809237  2.45675789]
 [ 1.79679623  3.10892238  0.46795304  2.29096375  4.89698074]
 [ 4.97169121  1.37596082  3.64235271  1.09233124 -4.20921877]
 [ 4.70602184  3.43671479  2.75438035  2.58356027  1.26078539]], 

  User matrix is [[ 1.88900214  0.96836741]
 [ 1.68228426  0.11916113]
 [ 1.07312416  1.47201824]
 [ 1.72127075  0.27153697]
 [-1.08313974  2.29298969]
 [ 0.7012941   1.71513796]], 

 Item matrix is [[ 0.65316147  1.59282075  0.01980163  1.16873018  2.91718381]
 [ 2.47674744  1.35247371  1.59782691  1.02845179 -0.45769986]], 

 Cost is 12.172405003118936




In [5]:
# 通过上面的结果观察，评分矩阵和实际的评分直接误差有点大。
# 这个时候思考可能分解的维度可能太低了，测试提高分解维度来观察结果。
K= 5
P, Q, cost = grad(R, K, max_iter, alpha, lamda)
predictR = np.dot(P, Q)
print("origin R is {}, \n\n predict Matrix is {}, \n\n  User matrix is {}, \n\n Item matrix is {}, \n\n Cost is {}\n\n".format(R, predictR, P, Q, cost))

origin R is [[4 0 1 0 5]
 [1 2 1 3 5]
 [4 5 3 1 0]
 [2 3 0 2 5]
 [5 1 4 0 0]
 [0 3 2 4 1]], 

 predict Matrix is [[3.86304464 4.52394822 1.23869478 3.33069646 4.99070213]
 [1.04329522 1.97974403 0.89772197 3.02042777 5.02443038]
 [4.05024057 5.00625101 2.91599961 1.00616638 7.58541172]
 [2.05075876 3.003719   1.1442414  1.95656048 5.00525261]
 [5.01028276 0.99793763 4.03090217 4.15424166 3.32771814]
 [4.69236137 3.00482005 1.89921073 4.02954803 1.01202584]], 

  User matrix is [[ 1.39249802  0.66321176  0.44565867  0.42395422  1.43196883]
 [ 0.34061424  1.79602222 -0.15002737  0.36029832  0.56774624]
 [ 1.71836154  0.31624823 -0.61982557  2.0647875   1.00663625]
 [ 0.95867661  0.92022398 -0.21956334  0.6476373   0.74396587]
 [ 0.58269231  0.78514465  1.64337465  1.57123616 -0.05634164]
 [ 0.29285693 -0.03167314  1.65964547  0.52963071  1.43693808]], 

 Item matrix is [[ 1.09304698  1.07500703  0.24179626  0.25569314  1.72154728]
 [ 0.02770414  0.16026387  0.27971541  1.47717672  1.8942

## 使用tensorflow来实现特征值分解

In [1]:

import tensorflow as tf
import numpy as np
R_square = np.array([[4,0,1,0,5],
     [1,2,1,3,5],
     [4,5,3,1,0],
     [2,3,0,2,5],
     [5,1,4,0,0]])

In [7]:
# 重新用矩阵的思维来写特征值分解。感觉以前写的有点问题。

import tensorflow as tf 

K=2
max_iter = 5000 #迭代次数多意味着步长比较小。
alpha = 0.0002
lamda = 0.004

def GDM_Tensor(R, K=2, max_iter= 5000, alpha=0.001, lamda= 0.002, cost_threshold = 0.0001):
    m = len(R)
    n = len(R[0])
    
    
    R_tf = tf.convert_to_tensor(R, dtype=float)
    P = tf.Variable(np.random.rand(m, K), dtype=float)
    Q = tf.Variable(np.random.rand(K, n), dtype=float)
    # temp = tf.matmul(P, Q)
    # print(temp)
    # Eui = tf.matmul(P, Q) - R_tf
    # print(Eui)
    for step in range(max_iter):
        Eui_matrix = tf.matmul(P, Q) - R_tf
        tf.matmul(tf.linalg.inv(tf.matmul(Q, tf.transpose(Q)) + lamda * tf.eye(K, K)), Q)
        # C = tf.square(Eui)
        
        # 这个是收敛条件之一。
        cost = 0
        cost = tf.reduce_sum(tf.reduce_sum(tf.square(Eui_matrix))) + lamda * (tf.reduce_sum(tf.reduce_sum(tf.square(P), 0)) + tf.reduce_sum(tf.reduce_sum(tf.square(Q), 0)))
        
        if cost <= cost_threshold:
            break
        
        pass
    pass

In [8]:
GDM_Tensor(R_square, K=3)

tf.Tensor(
[[1.3324616  1.343946   1.9704086  0.8768179  0.63844585]
 [1.0253268  0.51953644 1.4172212  0.8568473  0.44606578]
 [0.97683865 0.68724555 1.244671   0.7485718  0.40743855]
 [1.2816241  1.0012184  1.7197771  0.94676375 0.5596408 ]
 [1.3860558  1.385064   2.007086   0.9167477  0.65330523]], shape=(5, 5), dtype=float32)
tf.Tensor(
[[-2.6675384   1.343946    0.97040856  0.8768179  -4.361554  ]
 [ 0.02532685 -1.4804635   0.4172212  -2.1431527  -4.553934  ]
 [-3.0231614  -4.3127546  -1.755329   -0.2514282   0.40743855]
 [-0.7183759  -1.9987816   1.7197771  -1.0532362  -4.440359  ]
 [-3.613944    0.385064   -1.992914    0.9167477   0.65330523]], shape=(5, 5), dtype=float32)


In [7]:
# 使用tensorflow来实现这个函数.

import tensorflow as tf

def ModifyTensor(input_tensor, position=None, value=None):
    input_tensor = input_tensor.numpy()
    input_tensor[tuple(position)] = value
    return input_tensor

K=2
max_iter = 5000 #迭代次数多意味着步长比较小。
alpha = 0.0002
lamda = 0.004

# tf.debugging.set_log_device_placement(True)
def grad_tf(R, K=2, max_iter= 5000, alpha=0.001, lamda= 0.002, cost_threshold = 0.0001):
    R_tf = tf.convert_to_tensor(R, dtype=float)
    m = len(R)
    n = len(R[0])
    
    # P = np.random.rand(m, K)
    # Q = np.random.rand(K, n)
    P = tf.Variable(tf.zeros([m, K], dtype=float))
    Q = tf.Variable(tf.zeros([K, n], dtype=float))
    
    for step in range(max_iter):
        # 对所有的用户u和物品i做遍历。对对应的Pu和Qi向量进行梯度下降。
        for u in range(m):
            for i in range(n):
                # 对于每一个大于0的评分，求出评分误差。
                if R_tf[u][i] > 0:
                    eui = tf.matmul(tf.reshape(P[u, :], [1, -1]),tf.reshape(Q[:, i], [-1, 1])) - R_tf[u,i]
                    
                    # 带入梯度下降的公式，按照梯度下降算法更新当前的Pu和Qi。也就是按照K个隐藏维度来更新。
                    for k in range(K):
                        # 注意这里和公式不同的地方在于求和公式。由于求和是对i在求和，而本计算是包含在
                        # for i in range(n):当中的，就相对于每个步骤都减去了一个对于i的元素，所以不
                        # 用再求和了。
                        P = tf.py_function(ModifyTensor, 
                                           inp=[P, [u, k], P[u][k] - alpha * (2 * eui * Q[k][i] - 2 * lamda * P[u][k])], 
                                           Tout=P.dtype)
                        # 同样的
                        Q = tf.py_function(ModifyTensor, 
                                           inp=[Q, [k, i], Q[k][i] - alpha * (2 * eui * P[u][k] - 2 * lamda * Q[k][i])], 
                                           Tout=Q.dtype)
                
        # u和i遍历完成。所有特征向量都更新完成。可以计算预测评分矩阵。
        # predictR = np.dot(P, Q)
        # 计算当前的损失函数。
        cost = 0
        
        for u in range(m):
            for i in range(n):
                # 在评分矩阵R_tf中为0的不计算损失函数，原因依然是为0的评分可能是用户没有评分。
                if R_tf[u][i] > 0:
                    cost += (tf.matmul(tf.reshape(P[u, :], [1, -1]),tf.reshape(Q[:, i], [-1, 1])) - R_tf[u,i]) ** 2
                    for k in range(K):
                        cost += lamda * (tf.square(P[u][k]) + tf.square(Q[k][i]))
        # 当损失函数小于某一个特定阈值时退出。
        if cost < cost_threshold:
            break
    return P, Q, cost

In [12]:
P, Q, cost = grad_tf(R, K=3)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [8]:

R_tf = tf.convert_to_tensor(R_square, dtype=float)
# tensorflow进行eig必须是方阵。
# 
M, N = tf.eig(R_tf)


Executing op Eig in device /job:localhost/replica:0/task:0/device:CPU:0


In [9]:
print(M)

tf.Tensor(
[ 0.5489885-1.8442735e-08j -1.7649044-1.9651887e+00j
 -1.7649044+1.9651892e+00j  2.9616973-3.0399860e-08j
 11.019123 +6.4537744e-09j], shape=(5,), dtype=complex64)


In [10]:
print(N)

tf.Tensor(
[[-0.50880915+0.00434304j  0.28622308-0.30314255j -0.38530356+0.15924889j
  -0.56747574+0.20958522j  0.35919613-0.00087918j]
 [ 0.20886296-0.00178275j  0.05577823-0.29754078j -0.29786214-0.05403553j
   0.42365906-0.15646952j  0.4714589 -0.00115396j]
 [ 0.61511016-0.0052503j  -0.1284342 +0.6106717j   0.61630976+0.09786545j
   0.6015964 -0.2221869j   0.530147  -0.0012976j ]
 [-0.5167199 +0.00441057j  0.38827828+0.04287843j -0.09845588+0.37802777j
   0.12856518-0.04748289j  0.4572356 -0.00111915j]
 [ 0.22815934-0.00194746j -0.42346954+0.11488671j  0.25839487-0.35462296j
  -0.00247677+0.00091478j  0.39821923-0.00097469j]], shape=(5, 5), dtype=complex64)


In [11]:
x = tf.constant([[1,2], [3,4]])
y = tf.add(x, 1)
print(y)

Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)
